In [ ]:
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from google.cloud import monitoring_v3
from google.protobuf import duration_pb2
import sys
from pathlib import Path
from scipy.interpolate import interp1d

sys.path.append(str(Path.cwd().parent / 'cerulean_cloud') + "/")
from structured_logger import (
    log_query,
    query_logger,
    generate_log_file,
    get_scene_log_stats,
    get_latest_revision,
)

project_id = "cerulean-338116"
service_name = "cerulean-cloud-test-cr-orchestrator"
revision_name = "cerulean-cloud-test-cr-orchestrator-00101-k4q"

now = datetime.datetime.now()
end_time = now+datetime.timedelta(days=1)
start_time = now - datetime.timedelta(days=0.25)  # N days ago
tz_local = "US/Arizona"

project_name = f"projects/{project_id}"

# Initialize Cloud Monitoring client
client = monitoring_v3.MetricServiceClient()

In [ ]:


def fetch_memory_usage(revision_name, start_time, end_time, interval_min=5):
    """
    Fetch Cloud Run memory usage over the last 3 days.
    """
    metric_type = "run.googleapis.com/container/memory/utilizations"
    
    # Define query filter
    filt = f'''
        metric.type = "{metric_type}"
        AND resource.labels.revision_name = "{revision_name}"
    '''
    
    # Define aggregation settings
    aggregation=monitoring_v3.Aggregation(
        alignment_period=duration_pb2.Duration(seconds=interval_min*60),
        per_series_aligner=monitoring_v3.Aggregation.Aligner.ALIGN_PERCENTILE_50 
    )

    # Define query request
    query = monitoring_v3.ListTimeSeriesRequest(
        name=project_name,
        filter=filt,
        interval=monitoring_v3.TimeInterval(start_time=start_time, end_time=end_time),
        aggregation=aggregation,
        view=monitoring_v3.ListTimeSeriesRequest.TimeSeriesView.FULL,
    )

    try:
        results = client.list_time_series(query)
    except Exception as e:
        print(f"Error fetching memory usage for revision {revision_name}: {e}")
        return pd.DataFrame(columns=["timestamp", "memory_usage"])  # Return empty dataframe

    # Extract data points
    data = []
    for result in results:
        for point in result.points:
            timestamp = point.interval.end_time
            value = point.value.double_value if hasattr(point.value, "double_value") else None
            data.append([timestamp, value])

    return pd.DataFrame(data, columns=["timestamp", "memory_usage"])



def interp_logs_to_mem_ts(mem_ts, log_ts, mem):
    # ✅ Create interpolation function
    interp_func = interp1d(
        mem_ts.astype(np.int64) / 10**9,  # Convert timestamps to seconds
        mem,
        kind="linear",  # You can use "cubic" for smoother interpolation
        fill_value="extrapolate"  # Allow extrapolation if needed
    )
    
    return interp_func(log_ts.astype(np.int64) / 10**9)


def add_trace_hover(fig, df, size, color, name):

    
    fig.add_trace(go.Scatter(
        x=df['timestamp'], 
        y=df['y'], 
        mode='markers', 
        marker=dict(size=size, color=color),
        text=df['text'],  # Text displayed when hovering
        name=name,
        hoverinfo='text',  # Show text in hover
    ))

    return fig


def add_severity_trace(fig, hover_df):

    severities = ["INFO","WARNING","ERROR", "DEBUG"]
    colors = ["blue", "orange", "red", "green"]
    sizes = [3, 5, 10]

    
    tmp = hover_df[hover_df['mem_usage'].isnull()==False]
    fig.add_trace(go.Scatter(
        x=tmp['timestamp'], 
        y=tmp['y'], 
        mode='lines',
        line=dict(color='gray', width=1, dash='dash'),  # Customize line style
        name="Memory Usage",
        hoverinfo='skip'  # Disable hover interaction
    ))

    for severity, color, size in zip(severities, colors, sizes):

        tmp = hover_df[hover_df['severity']==severity]
        fig = add_trace_hover(fig, tmp, size, color, name=severity)
        
    return fig

def add_initiating_orchestrator_trace(fig, hover_df, size=5, color="green"):
    tmp = hover_df[hover_df['message'].str.contains("initiating orchestrator",case=False, na=False)]
    fig = add_trace_hover(fig, tmp, size, color, name="orchestrator initiaion")

    return fig


def plot_memory_usage(time, y, plot_type="matplotlib", hover_df=None, label="Memory Usage (%)"):
    if plot_type == "matplotlib":
        plt.figure(figsize=(10, 5))
        plt.plot(df_mem["timestamp"].dt.tz_convert("US/Arizona"), df_mem["memory_usage"], label=label)
        plt.xlabel("Timestamp")
        plt.ylabel("Memory Usage (bytes)")
        plt.title(f"Cloud Run Memory Usage for {revision_name}")
        plt.xticks(rotation=45)
        plt.legend()
        plt.show()
    elif plot_type =="plotly":
        # Create Scatter Plot with Clickable Points
        fig = go.Figure()
        
        fig.add_trace(go.Scatter(
            x=time, 
            y=y, 
            mode='lines',
            line=dict(color='black', width=2, dash='solid'),  # Customize line style
            name="Memory Usage",
            hoverinfo='skip'  # Disable hover interaction
        ))
        

        if hover_df is not None:
            fig = add_severity_trace(fig, hover_df)
            fig = add_initiating_orchestrator_trace(fig, hover_df, size=8, color="cyan")
            
        
        fig.update_layout(
            title=label,
            hovermode="closest",  # Enables point-specific hover interaction
            dragmode="zoom"
        )
        
        fig.show(config={"scrollZoom": True})

        return fig




In [ ]:

print("querying logger")
query = log_query(
    service_name,
    revision_name=revision_name,
    start_time=start_time,
    end_time=end_time,
)
logs = query_logger(project_id, query)
logs = logs.sort_values("timestamp").reset_index(drop=True)
logs["timestamp"] = logs["timestamp"].dt.tz_convert(tz_local)
logs["mem_usage"] = logs['json_payload'].apply(lambda x: x['perc_ram_used']/100 if x is not None and 'perc_ram_used' in x else None)
instance_id = logs['instanceId'].unique()[0]

print("querying metrics")
df_mem = fetch_memory_usage(revision_name, start_time, end_time, interval_min=1)
# df_mem = fetch_memory_usage(revision_name, instance_id=instance_id, start_time=start_time, end_time=end_time, interval_min=1)
df_mem["timestamp"] = df_mem["timestamp"].dt.tz_convert(tz_local)


# logs['y'] = interp_logs_to_mem_ts(df_mem['timestamp'], logs['timestamp'], df_mem["memory_usage"])

tmp = logs[logs['mem_usage'].isnull()==False]
# logs['y'] = interp_logs_to_mem_ts(tmp['timestamp'], logs['timestamp'], tmp["mem_usage"])
logs['y'] = logs['mem_usage'].fillna(0)




# Plot Memory Usage
if not df_mem.empty:
    hover_df = logs.copy()
    hover_df['text'] = hover_df['message']
    fig = plot_memory_usage(df_mem["timestamp"], df_mem["memory_usage"], plot_type="plotly", hover_df=hover_df)
else:
    print(f"No memory usage data found for revision {revision_name}")


In [ ]:
generate_log_file(logs, filename="log.txt")